In [1]:
##Import Required packages
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
%matplotlib inline
import pandas as pd
import numpy as np
import imageio
import os,sys
import pandas as pd
import re
import seaborn as sns
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [36]:
##Adding structured data
strucdf=pd.read_pickle('Final_XMLRevenuedataset.pkl')

#Only use window=1
strucdf['var1'] = strucdf['x'].apply(lambda x: x[0])
strucdf['var2'] = strucdf['x'].apply(lambda x: x[1])

# 10 Class Classificatoin
# Convert target into 10 categorical bins based on percentiles.
bin_labels = ['0', '1', '2', '3', '4','5','6','7','8','9']
strucdf['var3'] = pd.qcut(strucdf['y'],
                              q=[0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1],
                              labels=bin_labels)


# # 2 Class classification
# bin_labels = ['0', '1']
# strucdf['bins'] = pd.qcut(strucdf['y'],
#                               q=[0, .5, 1],
#                               labels=bin_labels)


# #4 class classification
# bin_labels = ['0', '1','2','3']
# strucdf['bins'] = pd.qcut(strucdf['y'],
#                               q=[0,.25, .5,.75, 1],
#                               labels=bin_labels)

strucdf.head()



strucdf['label']=strucdf['y'].apply(lambda x: 0 if x >= 0 else 1)
x= strucdf[['var1','var2']]
y = strucdf['y']
# y = strucdf['y']
print(strucdf.head())
strucdf.shape

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,shuffle=False)
y_train, y_test = y_train.astype(float), y_test.astype(float)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


print('------------------------------Class Distribution ----------------------------------------')
strucdf['var3'].value_counts()

,x,y,var1,var2,var3
0,"[0.026498422712933758, 0.10214285714285716]",-0.001077,0.026498,0.102143,3
1,"[-0.001076757421935087, 0.026498422712933758]",-0.220906,-0.001077,0.026498,0
2,"[-0.22090618537319048, -0.001076757421935087]",-0.018432,-0.220906,-0.001077,2
3,"[-0.0184320471860408, -0.22090618537319048]",0.001999,-0.018432,-0.220906,3
4,"[0.001999000499750125, -0.0184320471860408]",0.014258,0.001999,-0.018432,4


                                               x         y      var1  \
0    [0.026498422712933758, 0.10214285714285716] -0.001077  0.026498   
1  [-0.001076757421935087, 0.026498422712933758] -0.220906 -0.001077   
2  [-0.22090618537319048, -0.001076757421935087] -0.018432 -0.220906   
3    [-0.0184320471860408, -0.22090618537319048]  0.001999 -0.018432   
4    [0.001999000499750125, -0.0184320471860408]  0.014258  0.001999   

       var2 var3  label  
0  0.102143    3      1  
1  0.026498    0      1  
2 -0.001077    2      1  
3 -0.220906    3      0  
4 -0.018432    4      0  


(5674, 6)

------------------------------Class Distribution ----------------------------------------


9    568
6    568
3    568
0    568
8    567
7    567
5    567
4    567
2    567
1    567
Name: var3, dtype: int64

In [25]:
y_train

0      -0.001077
1      -0.220906
2      -0.018432
3       0.001999
4       0.014258
          ...   
4534    0.039028
4535    0.051070
4536   -0.008586
4537   -0.040137
4538    0.050600
Name: y, Length: 4539, dtype: float64

In [23]:
np.stack(X_train.values)[1]

array([-0.001076757421935087, 0.026498422712933758, '0'], dtype=object)

In [3]:
#bucket feature as a binary variable into logistic model.

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(np.stack(X_train.values),np.stack(y_train))
y_pred = logisticRegr.predict(np.stack(X_test.values))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [27]:
pd.DataFrame(y_train.values)[0].value_counts()

 0.000000    4
-1.000000    2
 1.000000    2
 0.745297    1
 0.042687    1
            ..
 0.042598    1
 0.587028    1
 0.043593    1
 0.624860    1
-0.220906    1
Name: 0, Length: 4534, dtype: int64

In [254]:
print('--------------------------y_pred------------------------------')
pd.DataFrame(y_pred)[0].value_counts()
print('--------------------------y_test------------------------------')
pd.DataFrame(y_test.values)[0].value_counts()

--------------------------y_pred------------------------------


0    661
1    474
Name: 0, dtype: int64

--------------------------y_test------------------------------


1    576
0    559
Name: 0, dtype: int64

**Logistic Regression Classifier**

In [255]:
from sklearn import metrics
count_misclassified = (y_test.values != y_pred).sum()
count_total = y_test.values.shape[0]
print('Misclassified test samples: {}'.format(count_misclassified))
print('Total test samples: {}'.format(count_total))
accuracy = metrics.accuracy_score(y_test.values, y_pred)
f1_score = metrics.f1_score(y_test.values, y_pred,average='weighted')
print('Accuracy: {:.2f}'.format(accuracy))
print('Weighted F1 Score: {:.2f}'.format(f1_score))


Misclassified test samples: 646
Total test samples: 1135
Accuracy: 0.43
Weighted F1 Score: 0.43


**Random Forest Classifier**

In [78]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
rf_model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# rf_model.fit(X_train.values.reshape(-1, 1), y_train.values.reshape(-1,1))
rf_model.fit(np.stack(X_train), np.stack(y_train))
print(f'Model Train Accuracy: {rf_model.score(np.stack(X_train), y_train.values.reshape(-1,1))}')
print(f'Model Test Accuracy: {rf_model.score(np.stack(X_test), y_test.values.reshape(-1,1))}')
rf_predictions = rf_model.predict(np.stack(X_test))
# Probabilities for each class
# rf_probs = rf_model.predict_proba(np.stack(X_test))[:, 1]

ValueError: Unknown label type: 'continuous'

In [ ]:
#k-fold CV for RF.
#hyperparameters for RF.

In [257]:
print('--------------------------y_pred------------------------------')
pd.DataFrame(rf_predictions)[0].value_counts()
print('--------------------------y_test------------------------------')
pd.DataFrame(y_test.values)[0].value_counts()

--------------------------y_pred------------------------------


1    594
0    541
Name: 0, dtype: int64

--------------------------y_test------------------------------


1    576
0    559
Name: 0, dtype: int64

In [250]:
from sklearn import metrics
count_misclassified = (y_test.values != rf_predictions).sum()
count_total = y_test.values.shape[0]
print('Misclassified test samples: {}'.format(count_misclassified))
print('Total test samples: {}'.format(count_total))
accuracy = metrics.accuracy_score(y_test.values, rf_predictions)
f1_score = metrics.f1_score(y_test.values, rf_predictions,average='weighted')
print('Accuracy: {:.2f}'.format(accuracy))
print('Weighted F1 Score: {:.2f}'.format(f1_score))


Misclassified test samples: 451
Total test samples: 1135
Accuracy: 0.60
Weighted F1 Score: 0.60


In [64]:
x.values

array([[-0.019679147750139236, -0.03634144365787506,
        0.018459182232618332, ..., 0.02011403048267155,
        0.032918917833256345, '7'],
       [-0.015117797069251537, -0.019679147750139236,
        0.02521229349076748, ..., 0.05956274276596148,
        0.02011403048267155, '6'],
       [-0.014624514617025852, -0.015117797069251537,
        0.030115768313407898, ..., 0.04638277584417983,
        0.05956274276596148, '4'],
       ...,
       [-0.29555097222328186, -0.28866058588027954, 0.03358547016978264,
        ..., 0.028393399024855943, -0.05055225148683093, '7'],
       [-0.2798287570476532, -0.29555097222328186, 0.047348007559776306,
        ..., 0.08316678192277588, 0.028393399024855943, '0'],
       [-0.2612355053424835, -0.2798287570476532, 0.04323828965425491,
        ..., -0.146683078309279, 0.08316678192277588, '9']], dtype=object)

In [75]:
from sklearn.model_selection import KFold # import KFold
kf = KFold(n_splits=2) # Define the split - into 2 folds 
kf.get_n_splits(x.values) # returns the number of splitting iterations in the cross-validator
print(kf) 
# KFold(n_splits=2, random_state=None, shuffle=False)

2

KFold(n_splits=2, random_state=None, shuffle=False)


In [77]:
for train_index, test_index in kf.split(x.values):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x.values[train_index], x.values[test_index]
    y_train, y_test = y[train_index], y[test_index]
    rf_model.fit(np.stack(X_train), np.stack(y_train))
    print(f'Model Train Accuracy: {rf_model.score(np.stack(X_train), y_train.values.reshape(-1,1))}')
    print(f'Model Test Accuracy: {rf_model.score(np.stack(X_test), y_test.values.reshape(-1,1))}')

TRAIN: [1596 1597 1598 ... 3189 3190 3191] TEST: [   0    1    2 ... 1593 1594 1595]


NameError: name 'rf_model' is not defined

In [ ]:
## Logistic Regression
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,f1_score,precision_score
reg = LogisticRegression().fit(np.stack(X_train),np.stack(y_train))
y_pred=reg.predict(np.stack(X_test))

#print(y_pred,y_test.to_numpy())
score=f1_score(y_pred,y_test)
print("F1 Score",score)
print(reg.precision_score)

**Linear Regression with category variable**

In [37]:
from sklearn.linear_model import LinearRegression
linearRegr = LinearRegression()  
linearRegr.fit(np.stack(X_train.values),np.stack(y_train))
y_pred = linearRegr.predict(np.stack(X_test.values))
        

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
linearRegr.coef_

array([-0.21529046, -0.09393125,  0.06096105])

In [30]:
linearRegr.intercept_

-0.22704834685905645

In [34]:
y_pred

array([ 0.05889547, -0.23865272, -0.14162734, ...,  0.13617142,
        0.12867295,  0.06387846])

In [35]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test.values, y_pred)

0.02067723970163462

**Without the category variable**

In [38]:
linearRegr.coef_

array([-0.30530215, -0.11687939])

In [39]:
linearRegr.intercept_

0.050099305074716514

In [40]:
y_pred

array([0.02431591, 0.03465278, 0.17167374, ..., 0.04645387, 0.03606253,
       0.03108925])

In [41]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test.values, y_pred)

0.051013790044730126

**Now including the sentiment scores into this**

In [42]:
df_2=pd.read_pickle('/mnt/nfs/scratch1/kajaltiwari/Merged/FinalMerged/Final_SentimentScore_TokenAvg.pkl')
df_2.head()

,x,y
0,"[-0.019679147750139236, -0.03634144365787506, ...",0.059563
1,"[-0.015117797069251537, -0.019679147750139236,...",0.046383
2,"[-0.014624514617025852, -0.015117797069251537,...",0.015211
3,"[-0.03730310872197151, -0.014624514617025852, ...",0.033951
4,"[-0.04820563271641731, -0.03730310872197151, -...",-0.017975


In [43]:
df_2['x'][0]

array([-0.01967915, -0.03634144,  0.01845918,  0.00939133,  0.02011403,
        0.03291892])

In [52]:
##Adding structured data
strucdf=pd.read_pickle('/mnt/nfs/scratch1/kajaltiwari/Merged/FinalMerged/Final_SentimentScore_TokenAvg.pkl')

#Only use window=1
strucdf['var1'] = strucdf['x'].apply(lambda x: x[0])
strucdf['var2'] = strucdf['x'].apply(lambda x: x[1])
strucdf['var3'] = strucdf['x'].apply(lambda x: x[2])
strucdf['var4'] = strucdf['x'].apply(lambda x: x[3])
strucdf['var5'] = strucdf['x'].apply(lambda x: x[4])
strucdf['var6'] = strucdf['x'].apply(lambda x: x[5])

# 10 Class Classificatoin
# Convert target into 10 categorical bins based on percentiles.
bin_labels = ['0', '1', '2', '3', '4','5','6','7','8','9']
strucdf['var7'] = pd.qcut(strucdf['y'],
                              q=[0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1],
                              labels=bin_labels)


# # 2 Class classification
# bin_labels = ['0', '1']
# strucdf['bins'] = pd.qcut(strucdf['y'],
#                               q=[0, .5, 1],
#                               labels=bin_labels)


# #4 class classification
# bin_labels = ['0', '1','2','3']
# strucdf['bins'] = pd.qcut(strucdf['y'],
#                               q=[0,.25, .5,.75, 1],
#                               labels=bin_labels)

strucdf.head()



strucdf['label']=strucdf['y'].apply(lambda x: 0 if x >= 0 else 1)
x= strucdf[['var1','var2','var3','var4','var5','var6','var7']]
y = strucdf['y']
# y = strucdf['y']
print(strucdf.head())
strucdf.shape

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,shuffle=False)
y_train, y_test = y_train.astype(float), y_test.astype(float)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


print('------------------------------Class Distribution ----------------------------------------')
strucdf['var3'].value_counts()

,x,y,var1,var2,var3,var4,var5,var6,var7
0,"[-0.019679147750139236, -0.03634144365787506, ...",0.059563,-0.019679,-0.036341,0.018459,0.009391,0.020114,0.032919,7
1,"[-0.015117797069251537, -0.019679147750139236,...",0.046383,-0.015118,-0.019679,0.025212,0.018459,0.059563,0.020114,6
2,"[-0.014624514617025852, -0.015117797069251537,...",0.015211,-0.014625,-0.015118,0.030116,0.025212,0.046383,0.059563,4
3,"[-0.03730310872197151, -0.014624514617025852, ...",0.033951,-0.037303,-0.014625,0.002744,0.030116,0.015211,0.046383,5
4,"[-0.04820563271641731, -0.03730310872197151, -...",-0.017975,-0.048206,-0.037303,-0.000949,0.002744,0.033951,0.015211,2


                                                   x         y      var1  \
0  [-0.019679147750139236, -0.03634144365787506, ...  0.059563 -0.019679   
1  [-0.015117797069251537, -0.019679147750139236,...  0.046383 -0.015118   
2  [-0.014624514617025852, -0.015117797069251537,...  0.015211 -0.014625   
3  [-0.03730310872197151, -0.014624514617025852, ...  0.033951 -0.037303   
4  [-0.04820563271641731, -0.03730310872197151, -... -0.017975 -0.048206   

       var2      var3      var4      var5      var6 var7  label  
0 -0.036341  0.018459  0.009391  0.020114  0.032919    7      0  
1 -0.019679  0.025212  0.018459  0.059563  0.020114    6      0  
2 -0.015118  0.030116  0.025212  0.046383  0.059563    4      0  
3 -0.014625  0.002744  0.030116  0.015211  0.046383    5      0  
4 -0.037303 -0.000949  0.002744  0.033951  0.015211    2      1  


(3192, 10)

------------------------------Class Distribution ----------------------------------------


0.026431    6
0.013328    5
0.013222    5
0.023434    4
0.032021    3
           ..
0.055015    1
0.045587    1
0.022798    1
0.177089    1
0.057959    1
Name: var3, Length: 3156, dtype: int64

In [53]:
X_train

,var1,var2,var3,var4,var5,var6,var7
0,-0.019679,-0.036341,0.018459,0.009391,0.020114,0.032919,7
1,-0.015118,-0.019679,0.025212,0.018459,0.059563,0.020114,6
2,-0.014625,-0.015118,0.030116,0.025212,0.046383,0.059563,4
3,-0.037303,-0.014625,0.002744,0.030116,0.015211,0.046383,5
4,-0.048206,-0.037303,-0.000949,0.002744,0.033951,0.015211,2
...,...,...,...,...,...,...,...
2548,-0.192503,-0.175809,-0.106001,-0.118385,-0.001524,-0.340669,9
2549,-0.220485,-0.192503,-0.160428,-0.106001,0.290369,-0.001524,1
2550,-0.213561,-0.220485,-0.140434,-0.160428,-0.094825,0.290369,3
2551,-0.167046,0.030724,-0.035101,0.050066,-0.024455,-0.048247,1


In [54]:
from sklearn.linear_model import LinearRegression
linearRegr = LinearRegression()  
linearRegr.fit(np.stack(X_train.values),np.stack(y_train))
y_pred = linearRegr.predict(np.stack(X_test.values))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [49]:
linearRegr.coef_

array([-0.12292396,  0.16327975,  0.45286299, -0.09428932, -0.30346621,
       -0.14199952])

In [50]:
linearRegr.intercept_

0.042536904787850396

In [51]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test.values, y_pred)

0.04284610395395106

**Including the sentiment score and cat variable**

In [56]:
linearRegr.coef_

array([-0.13102293,  0.08550388, -0.17421047,  0.11420401, -0.20099868,
       -0.09943581,  0.06229367])

In [57]:
linearRegr.intercept_

-0.24471936678623676

In [58]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test.values, y_pred)

0.02118402539888656